In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Install required packages

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     |████████████████████████████████| 4.9 MB 19.0 MB/s 
     |████████████████████████████████| 163 kB 49.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.4 MB/s 
     |████████████████████████████████| 53 kB 760 kB/s 
     |████████████████████████████████| 175 kB 11.3 MB/s 
     |████████████████████████████████| 235 kB 44.6 MB/s 


In [ ]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 28.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [ ]:
import json
f = open('/content/drive/MyDrive/car_data/tondar90.json' , encoding='utf-8', errors='ignore')
import subprocess as sp
from pandas.core.common import flatten

data = json.load(f)
s1=[]
for i in range(0,len(data['messages'])):
  if len(data['messages'][i]['text']) >= 1 :
    
    if len(data['messages'][i]['text'][0]) == 1 :
       s1.append(data['messages'][i]['text'])
      

  


 


s0=list(flatten(s1))


In [ ]:
mydict={
    "اصتحلاک" :"اصطکاک"
    ,
    "اصتحهلاک": "استهلاک"
    ,
    "اصحلاک" : "استهلاک"
    ,
    "اصطحلاک" :"استهلاک"
    ,
    "اصطلاک":"استهلاک"
    ,
    "اصطکاک":"اصطکاک"
    ,
    "انزکتور": "انژکتور"
    ,
    "بربرینگ":"بربلینگ"
    ,
    "دفرانسیل":"دیفرانسیل"
    ,
    "میکن":"میگن"
    ,
    "یسری":"یه سری"
  
    

}

In [ ]:
sentences=[]
for sentence in s0:
  for word in mydict:
     sentence=sentence.replace(word,mydict[word])
  sentences.append(sentence)



In [ ]:
s0=sentences

In [ ]:
len(s0)

480874

In [ ]:
from __future__ import unicode_literals
from hazm import *
import re
t=[]
charRe =  re.compile("[.;:!\'?,\"()\[\]]")
charRe1 =  re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
i=0
qq=[]
for index,item in enumerate(s0): 
  d1 = re.sub('[%�.()-=+*/,!]', '', item)
  t.append(d1)

m=["?","؟"]
for s in s0:
   if (any(x in s for x in m)==False):
      qq.append(s)

for index,item in enumerate(qq): 
  charRe.match(item)
  charRe1.match(item)

t=qq
   

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
import pandas as pd

data1 = pd.DataFrame (qq, columns = ['comment'])

In [ ]:
data1['cleaned_comment'] = data1['comment'].apply(cleaning)

In [ ]:
# print data information
print('data information')
print(data1.info(), '\n')

# print missing values information
print('missing values stats')
print(data1.isnull().sum(), '\n')



data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390732 entries, 0 to 390731
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   comment          390732 non-null  object
 1   cleaned_comment  390732 non-null  object
dtypes: object(2)
memory usage: 6.0+ MB
None 

missing values stats
comment            0
cleaned_comment    0
dtype: int64 



In [ ]:
# handle some conflicts with the dataset structure
# you can find a reliable solution, for the sake of the simplicity
# I just remove these bad combinations!

data1 = data1.dropna(subset=['cleaned_comment'])
data1 = data1.drop_duplicates(subset=['cleaned_comment'], keep='first')
data1 = data1.reset_index(drop=True)


# previous information after solving the conflicts

# print data information
print('data information')
print(data1.info(), '\n')

# print missing values information
print('missing values stats')
print(data1.isnull().sum(), '\n')


data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325780 entries, 0 to 325779
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   comment          325780 non-null  object
 1   cleaned_comment  325780 non-null  object
dtypes: object(2)
memory usage: 5.0+ MB
None 

missing values stats
comment            0
cleaned_comment    0
dtype: int64 



In [ ]:
dd=[]
matches=["گیربکس","دیفرانسیل","دفرانسیل","بلبرینگ","توربو","ابروغن","گیج ","میل لنگ","تسمه تایم","سوپاپ","سیلندر","شمع","وایر","جرقه","احتراق","کمپرسور","پروانه","بازویی","دمنده","رادیاات","موتور","سوپاپ","اسب بخار","اسببخار","گشتاور","سرعت","سرعت","اگزوز","قدرت","نیوتون ","شتاب ","مصرف ترکیبی","روغن ","ابرادیاتور ","اب رادیاتور","فیلتر ","اتیلن ","کاتالیزور ","گاز ","مقطر ","روغن هیدرولیک ","بدنه","شاسی","اتاق","محور","باک","لاستیک","ارتفاع","عرض","طول","وزن","سنگین","سبک","سدان","یکپارچه","مستقل ","حجم","عقب ","جلو","استپ","ایستایی","سربالایی","سرازیری","کشش","پایداری","کنترل ","تطبیقی","سرعت","واژگونی","ضد قفل","ای بی اس","چرخ","تایر","ترمز","ترمز دستی","پدال","برف","باران","بارون","بارندگی","توزیع ","نیرو ","اسفالت ","کاسه نمد ","دیسک ","استهلاک ","اصطکاک","استحکام","چراغ ترمز","مه شکن","نوربالا","چراغ جلو","پرژکتور","دی لایت","چراغ","فرمان","نیرو","کمکی","صوتی","صدا ","بلند گو","ضبط ","اهنگ","کیسه هوا","ایربگ","کمربند","تهویه","کولر","بخاری","خنک کننده","ترموستات","فن","صندلی","پارچه","چرم","روکش","نمد","ابشن ","سیگار","فندک","داشبورد","کنسول","چراغ مطالعه","داشبرد","شیشه","سانروف","دودی","شیشه شور","آینه ","آیینه","تاشو ","پاناروما","رنگ","کهکشانی","زنگ","دو پوسته","متالیک","اسپورت","کوسه ای","انتن","دفیوزر","رکاب","رینگ","لوتوس","قاب","لگو","بدنه","گلگیر","سپر","کاپوت","برف پاکن","شلگیر","دستگیره","پلاک","آژیر","اداپتور","باطری","اتصال","برق","ارور","سنسور","آمپر","امپر","قفل فرمون","زنجیر","صندلی کودک","قفل پدال","آچار","اچار","کمک","جعبه کمک","اولیه","جک","یدکش","بکسل","زاپاس","امداد خودرو","نمایندگی","تعمیرگاه","کارخونه","کارخانه",]
for sentence in data1.cleaned_comment:
   if any(x in sentence for x in matches):
      dd.append(sentence)

In [ ]:
import pandas as pd

Main_data = pd.DataFrame (dd, columns = ['comment'])
Main_data

,comment
0,عوضش لاستیک ژاپنی تا حداقل ۵ سال خیالت راحت با...
1,۲۰۵ فرمان پذیری رو کم می‌کنه و فرمون سفت. می ش...
2,ظاهر بیرونی ۲۰۷ که همون ۲۰۶ تکراریه. استپ خیلی...
3,بابا استپ سالاره قاطی مرغا نزنیدش
4,از ۹۱ تا الان ۲۰۵ زیرشه آخ نگفته از ۱۸۵ هم واق...
...,...
107363,والا نمیدونم، اگه اصل باشه هر ۲خوبن، trw دیسک ...
107364,این که عکس روغن منه
107365,ماه پیش trw اصلی را با لنت رنو فابریک بستم، لا...
107366,سلام دوستان من الان خودم تهرانم ولی برام پیامک...


In [ ]:
qq=[]
m=["چه جوری","چی ","چطوره","چطوریه","برای چی","به چه علت","چرا ","چیه","چجوری"]
for s in Main_data.comment:
   if (any(x in s for x in m)==False):
      qq.append(s)

In [ ]:
import pandas as pd

Main_data2 = pd.DataFrame (qq, columns = ['cleaned_comment'])
Main_data2


,cleaned_comment
0,عوضش لاستیک ژاپنی تا حداقل ۵ سال خیالت راحت با...
1,۲۰۵ فرمان پذیری رو کم می‌کنه و فرمون سفت. می ش...
2,ظاهر بیرونی ۲۰۷ که همون ۲۰۶ تکراریه. استپ خیلی...
3,بابا استپ سالاره قاطی مرغا نزنیدش
4,از ۹۱ تا الان ۲۰۵ زیرشه آخ نگفته از ۱۸۵ هم واق...
...,...
101400,والا نمیدونم، اگه اصل باشه هر ۲خوبن، trw دیسک ...
101401,این که عکس روغن منه
101402,ماه پیش trw اصلی را با لنت رنو فابریک بستم، لا...
101403,سلام دوستان من الان خودم تهرانم ولی برام پیامک...


In [ ]:
 Main_data2.sample(n=100)

,cleaned_comment
83408,یه دیاگ بزن بگو شمع ماشین رو هم تو دیاگ چک کنه
61746,پلاستیکی احتمال تقلبی بودنش بالاست، همین
13520,سلام. منم ۱/۷۰۰سودتاخیرگرفتم. بایدهرردزبزنگی ب...
63930,به جز اینا سمند یکی از همکاران سوییچ مونده بود...
21268,منظورم این بود.. اگزوز رو عوض نکردن.. هی روغن ...
...,...
65365,دیروز ادینول از نمایندگی رفتم شخصا خریدم ۴لیتر...
60521,نه بابا همون لوله‌ای که گیج میره داخلش رو باز ...
20975,یبار پشت فرمونش بشینید …گاز خورش عالیه
28662,داشبورد تندر مثل داشبورد پرایده


In [ ]:
len(Main_data2)

101405

In [ ]:
import pickle 
with open('/content/drive/MyDrive/pickle file/tondar90.pkl', 'wb') as handle:
    pickle.dump(Main_data2, handle, protocol=pickle.HIGHEST_PROTOCOL)